In [1]:
#!pip install requirements.txt

In [ ]:
#!bash load_data.sh

В этом ноутбуке также решается задача NER как задача разметки последовательности, но в качестве архитектуры выбран  предобученный BERT + Softmax из библиотеки HuggingFace. Так как библиотека предоставляет свой фреймворк для обучения, было решено использовать фреймворк HF. Самая главная задача - прикрутить токензатор BERT-а.

In [14]:
import os, sys

In [15]:
sys.path.append(os.path.join(os.getcwd(), 'ner'))

In [3]:
import torch

In [4]:
USE_GPU = torch.cuda.is_available()

In [5]:
USE_GPU = False

In [20]:
DO_FINETUNE = False

In [6]:
PATH_TO_TRAIN = './conll2003/train.txt'
PATH_TO_TEST = './conll2003/test.txt'
PATH_TO_VALID = './conll2003/valid.txt'

In [7]:
from ner.data_utils import load_sentences, iter_over_sentences, iter_over_words, iter_over_tags, create_mapping

In [8]:
train_sentences = load_sentences(PATH_TO_TRAIN)
test_sentences = load_sentences(PATH_TO_TEST)
valid_sentences = load_sentences(PATH_TO_VALID)

In [9]:
from transformers import AutoTokenizer

MODEL_CHECKPOINT = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(MODEL_CHECKPOINT)

In [10]:
from ner.data_utils import create_mapping, iter_over_tags

In [11]:
tag_to_id, id_to_tag = create_mapping(train_sentences, iter_over_tags, False)

In [12]:
from ner.datasets.hf_bert_dataset import create_bert_dataset

In [13]:
train_data = create_bert_dataset(train_sentences, tokenizer, tag_to_id)
test_data = create_bert_dataset(test_sentences, tokenizer, tag_to_id)
valid_data = create_bert_dataset(valid_sentences, tokenizer, tag_to_id)

In [21]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [6]:
import torch
import torch.nn as nn

from datasets import Dataset

from torch import autograd
from torch.autograd import Variable
from torch.nn import init
from torch.utils.data import DataLoader, TensorDataset
from torch.nn.utils.rnn import pad_sequence

from transformers import AutoConfig, BertModel, BertTokenizerFast, BertPreTrainedModel

In [16]:
from ner.metrics.ner import compute_metrics

In [17]:
metrics_computer = lambda eval_preds: compute_metrics(eval_preds, id_to_tag)

In [19]:
from transformers import AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained(
    MODEL_CHECKPOINT,
    id2label=id_to_tag,
    label2id=tag_to_id,
)

NOTE: Redirects are currently not supported in Windows or MacOs.
Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForTokenClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassificatio

In [21]:
if DO_FINETUNE:
    for param in model.bert.parameters():
          param.requires_grad = False

In [22]:
from transformers import TrainingArguments

args = TrainingArguments(
    "bert-connl",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=10,
    weight_decay=0.01,
    push_to_hub=False,
    logging_steps=10
)

In [35]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_data,
    eval_dataset=valid_data,
    data_collator=data_collator,
    compute_metrics=metrics_computer,
    tokenizer=tokenizer,
)
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 14041
  Num Epochs = 10
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 17560


Epoch,Training Loss,Validation Loss,Spans precision,Spans recall,Spans f1,Spans accuracy,Token o precision,Token o recall,Token o f1-score,Token o support,Token per precision,Token per recall,Token per f1-score,Token per support,Token org precision,Token org recall,Token org f1-score,Token org support,Token loc precision,Token loc recall,Token loc f1-score,Token loc support,Token misc precision,Token misc recall,Token misc f1-score,Token misc support,Token macro avg precision,Token macro avg recall,Token macro avg f1-score,Token macro avg support,Token weighted avg precision,Token weighted avg recall,Token weighted avg f1-score,Token weighted avg support
1,0.101500,0.060751,0.934970,0.944529,0.939725,0.983635,0.982915,0.916833,0.948725,4016,0.908770,0.870594,0.889273,2071,0.992684,0.996868,0.994772,51723,0.934685,0.938641,0.936659,3781,0.963117,0.981752,0.972346,6357,0.956434,0.940937,0.948355,67948,0.983555,0.983635,0.983503,67948
2,0.015700,0.064469,0.937699,0.962584,0.949979,0.986151,0.960634,0.966135,0.963377,4016,0.907146,0.901014,0.904070,2071,0.995574,0.995843,0.995708,51723,0.954496,0.920920,0.937407,3781,0.969692,0.986472,0.978010,6357,0.957508,0.954077,0.955714,67948,0.986106,0.986151,0.986104,67948
3,0.074200,0.061128,0.937301,0.959321,0.948183,0.985842,0.970149,0.954930,0.962480,4016,0.857464,0.920811,0.888009,2071,0.994979,0.996075,0.995527,51723,0.956164,0.923036,0.939308,3781,0.983126,0.980651,0.981887,6357,0.952376,0.955101,0.953442,67948,0.986051,0.985842,0.985892,67948
4,0.007500,0.065547,0.948183,0.959321,0.953720,0.986490,0.973860,0.946215,0.959838,4016,0.883504,0.915500,0.899217,2071,0.994942,0.996365,0.995653,51723,0.938210,0.939698,0.938953,3781,0.988917,0.982539,0.985718,6357,0.955886,0.956064,0.955876,67948,0.986579,0.986490,0.986512,67948
5,0.000200,0.074092,0.946287,0.961932,0.954045,0.987638,0.966220,0.968625,0.967421,4016,0.925797,0.897634,0.911498,2071,0.995116,0.996675,0.995895,51723,0.943456,0.939963,0.941706,3781,0.985969,0.983797,0.984882,6357,0.963312,0.957339,0.960280,67948,0.987565,0.987638,0.987594,67948
6,0.014100,0.098156,0.959991,0.955188,0.957584,0.985754,0.984461,0.930777,0.956867,4016,0.922000,0.890391,0.905920,2071,0.995825,0.996017,0.995921,51723,0.886014,0.960063,0.921554,3781,0.988926,0.983325,0.986118,6357,0.955445,0.952115,0.953276,67948,0.986147,0.985754,0.985814,67948
7,0.000800,0.089398,0.950633,0.963454,0.957001,0.987652,0.982573,0.954681,0.968426,4016,0.883356,0.925157,0.903774,2071,0.995847,0.996829,0.996338,51723,0.945751,0.935996,0.940848,3781,0.984279,0.984899,0.984589,6357,0.958361,0.959512,0.958795,67948,0.987764,0.987652,0.987680,67948
8,0.000100,0.090718,0.954055,0.962149,0.958085,0.987461,0.972426,0.957171,0.964738,4016,0.917969,0.907774,0.912843,2071,0.995732,0.996791,0.996261,51723,0.927476,0.943666,0.935501,3781,0.988137,0.982696,0.985409,6357,0.960348,0.957620,0.958950,67948,0.987475,0.987461,0.987459,67948
9,0.000000,0.089757,0.953518,0.963889,0.958676,0.988138,0.973374,0.964890,0.969113,4016,0.916626,0.907774,0.912179,2071,0.995444,0.996926,0.996184,51723,0.943326,0.942079,0.942702,3781,0.987539,0.984899,0.986217,6357,0.963262,0.959314,0.961279,67948,0.988098,0.988138,0.988115,67948
10,0.000000,0.093044,0.955172,0.964107,0.959619,0.987844,0.977648,0.958416,0.967937,4016,0.913359,0.911154,0.912255,2071,0.995961,0.996481,0.996221,51723,0.931564,0.946839,0.939140,3781,0.986304,0.985528,0.985915,6357,0.960967,0.959684,0.960294,67948,0.987874,0.987844,0.987850,67948


***** Running Evaluation *****
  Num examples = 3250
  Batch size = 8
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: ORG seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: LOC seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: MISC seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: PER seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
Saving model checkpoint to bert-connl/checkpoint-1756
Configuration saved in bert-connl/checkpoint-1756/config.json
Model weights saved in bert-connl/checkpoint-1756/pytorch_model.bin
tokenizer config 

TrainOutput(global_step=17560, training_loss=0.021032569046603927, metrics={'train_runtime': 2099.0282, 'train_samples_per_second': 66.893, 'train_steps_per_second': 8.366, 'total_flos': 3075586560363330.0, 'train_loss': 0.021032569046603927, 'epoch': 10.0})

In [36]:
trainer.evaluate(test_data)

***** Running Evaluation *****
  Num examples = 3453
  Batch size = 8


/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: LOC seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: PER seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: MISC seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: ORG seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


{'eval_loss': 0.2676807940006256,
 'eval_spans precision': 0.9065281899109793,
 'eval_spans recall': 0.9229607250755287,
 'eval_spans f1': 0.9146706586826349,
 'eval_spans accuracy': 0.9736373520745024,
 'eval_token O precision': 0.9224037339556592,
 'eval_token O recall': 0.9256440281030445,
 'eval_token O f1-score': 0.9240210403272939,
 'eval_token O support': 3416,
 'eval_token PER precision': 0.7261833433193529,
 'eval_token PER recall': 0.7632241813602015,
 'eval_token PER f1-score': 0.7442431685600246,
 'eval_token PER support': 1588,
 'eval_token ORG precision': 0.9937240250199404,
 'eval_token ORG recall': 0.9878560250391236,
 'eval_token ORG f1-score': 0.9907813366538658,
 'eval_token ORG support': 47925,
 'eval_token LOC precision': 0.9028876582278481,
 'eval_token LOC recall': 0.9445478998551624,
 'eval_token LOC f1-score': 0.9232480533926586,
 'eval_token LOC support': 4833,
 'eval_token MISC precision': 0.9717613836921991,
 'eval_token MISC recall': 0.9661344095455343,
 'e